<a href="https://colab.research.google.com/github/Nawaf9997/test_t5/blob/main/YOLO_Practical_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [80]:
# prompt: library opencv

!pip install opencv-python


In [81]:
!pip install ultralytics -q
!pip install opencv-python

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [82]:
import cv2
from ultralytics import YOLO

In [83]:
model_path = '/content/YOLO_Model.pt'
model = YOLO(model_path)

print("YOLO model loaded successfully.")

YOLO model loaded successfully.


## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [84]:
video_path = '/content/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [85]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Print the results
print(f"Video Width: {width}")
print(f"Video Height: {height}")
print(f"Frames Per Second (FPS): {fps}")

Video Width: 1920
Video Height: 1080
Frames Per Second (FPS): 29


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [86]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_path = 'output_2.avi'
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [87]:
your_name = "Nawaf"
left_count = 0
right_count = 0
top_count = 0
bottom_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]
    results = model(frame)

    for result in results:
        for detection in result.boxes.data:
            x1, y1, x2, y2, score, class_id = detection[:6]
            class_id = int(class_id)

            if score > 0.5:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                # Draw a secondary bounding box slightly inside the first one
                cv2.rectangle(frame, (int(x1)+5, int(y1)+5), (int(x2)-5, int(y2)-5), (255, 0, 0), 2)

                # Add a label with the class ID and score
                label = f'ID: {class_id} Score: {score:.2f}'
                cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                center_x = (x1 + x2) / 2
                center_y = (y1 + y2) / 2

                if center_x < width / 2 and center_y > height / 2:
                    left_count += 1
                elif center_x > width / 2 and center_y > height / 2:
                    right_count += 1
                elif center_y < height / 2 and center_x > width / 2:
                    top_count += 1
                elif center_y > height / 2 and center_x < width / 2:
                    bottom_count += 1

    # Display the current counts for each direction and my name
    cv2.putText(frame, your_name, (width // 2 - 100, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 3)
    cv2.putText(frame, f'Left Count: {left_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(frame, f'Right Count: {right_count}', (width - 300, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(frame, f'Top Count: {top_count}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(frame, f'Bottom Count: {bottom_count}', (width - 300, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 2870.5ms
Speed: 8.0ms preprocess, 2870.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 3 vans, 2532.1ms
Speed: 4.3ms preprocess, 2532.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 2516.6ms
Speed: 5.1ms preprocess, 2516.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 2842.2ms
Speed: 8.0ms preprocess, 2842.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 3692.9ms
Speed: 7.8ms preprocess, 3692.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 2569.6ms
Speed: 4.2ms preprocess, 2569.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 2531.2ms
Speed: 5.0ms p

KeyboardInterrupt: 

## Save and Submit Your Work

Submit both the notebook and the output video